## **APPLYING LEIDEN ALGORITHM**

In [1]:
import numpy as np
import re
import os
import leidenalg
import igraph as ig
import matplotlib.pyplot as plt
from sklearn.cluster import AgglomerativeClustering
import scipy.io
from sklearn.neighbors import NearestNeighbors 
dataset='sunny'   #change dataset here !!!

**CLUSTERING USING OVERLAP**

# ---LOCALIZATION based on average descriptor---

In [16]:

os.chdir("/home/hades/codes/leiden/cold")
mat = scipy.io.loadmat('%s_VideoTable_FR3_1_300_DEFAULT.mat'%dataset)
#mat2 = scipy.io.loadmat('matrix_testing.mat') # FOR TESTING ONLY
os.chdir("/home/hades/codes/leiden/localization/")
import cv2
#I=cv2.imread('test.jpg')


Load main dataset descriptors:

In [17]:
descriptor=np.empty((len(f),98128),float)
for i in range(0,len(f)-2):
    descriptor[i]=mat['VideoTable_fr_300'][0][i][0][0]

descriptor2=np.empty((len(f),98128),float)
for i in range(0,len(f)-2):
    descriptor2[i]=mat2['VideoTable_fr_300'][0][i][0][0]

avg=np.empty((len(final_Clusters),98128),float)
for i in range(0,len(final_Clusters)):
    suma=np.sum(descriptor[final_Clusters[i][:]],axis=0)
    #print(x)
    l=len(final_Clusters[i][:])
    avg[i]=suma/l
    
#den xreiazetai 

### **Localization with K-NN**
 
**Finding test sample's nearest neighboors**

descriptor_test=descriptor2[1]#225

neigh = NearestNeighbors(n_neighbors=10)
neigh.fit(descriptor) 
print(neigh.kneighbors(descriptor_test.reshape(1, -1))) 
closest=neigh.kneighbors(descriptor_test.reshape(1, -1))



**Classifying based on closest neighbors cluster**

a=np.empty((len(final_Clusters),1),int).tolist()
for i in range(0,len(final_Clusters)):
    a[i]=np.asarray(final_Clusters[i])   
    
closest_vector=[]
for k in closest[1][0]:
    #print('k=',k)
    for i in range(0,len(final_Clusters)):
        #print(np.argwhere(k==final_Clusters[i]))
        #print(np.nonzero(k==final_Clusters[i]))
        if np.any(np.nonzero(k==final_Clusters[i])):
            closest_vector.append(i)
            #print(i)
#print(closest_vector)    
counter=np.zeros((len(final_Clusters),1),int)    
for i in range(0,len(closest_vector)):
    counter[closest_vector[i]]+=1
print(counter)
print("The test samlple is in cluster " ,np.argmax(counter,axis=0)[0])    

### END ###

In [18]:
#import cairocffi as cairo
import math
G1=G
from sklearn.preprocessing import normalize

print("edges in original graph :", G1.ecount())
edge_list=G1.es.select(weight_lt=300) 
G1.delete_edges(edge_list)
print("edges after removal :",G1.ecount())
G1.closeness( normalized=True)
#G1.evcent(directed=False,scale=True,weights=None,return_eigenvalue=False)
total_score=[]

best_centrality=[]
print(len(final_Clusters))
for i in range(0,len(final_Clusters)):
    print('interiation no ',i,'\n')
    h=G1.subgraph(final_Clusters[i])
    ig.plot(h)
    temp_closeness=h.closeness(normalized=True)
   # print(np.array(temp_closeness) , '\n')
    temp_betweeness=h.betweenness(vertices=None, directed=False, cutoff=None, weights=None)
    #print(np.array(temp_betweeness),'\n')
    
    total_score.append([])
    nan_check=0
    
    if np.any(temp_closeness):
        nan_check=math.isnan(temp_closeness[0])
        print('is nan')
    if(np.any(temp_closeness)) and not (nan_check) :
        #print (temp_closeness)
        
        temp_closeness_norm=normalize(np.array(temp_closeness).reshape(-1,1),axis=0)
        #print(temp_closeness)
        temp_betweeness_norm=normalize(np.array(temp_closeness).reshape(-1,1),axis=0)
    for k in range(0,len(final_Clusters[i])):
        total_score[i].append(0.5*temp_closeness_norm[k]+0.5*temp_betweeness_norm[k])
      #  print(total_score)
    #print (total_score[i])
    #print(temp_closeness)
    if(np.any(temp_closeness)):
        best_centrality.append(np.argmax(np.array(total_score[i])))
    else:
        best_centrality.append([])   
#print(normalize(np.array(temp_closeness).reshape(-1,1),axis=0))
print(best_centrality)
best_frame=[]
best_frame_descriptor=[]
best_frame_local=[]
for i in range(0,len(final_Clusters)):    
    
        #best_frame.append([])
    best_frame.append(final_Clusters[i][best_centrality[i]])
    best_frame_descriptor.append(descriptor[final_Clusters[i][best_centrality[i]]])#(descriptor[best_frame[i]])
    best_frame_local.append(best_centrality[i])    
        #best_frame[i].extend([i])
    #else:
    #    best_frame.append(-1)
    #    best_frame_descriptor.append(-1)
    # ( best_frame , cluster that belongs )
    
#best_frame
#best_frame_descriptor

edges in original graph : 1276003
edges after removal : 62373
11
interiation no  0 

is nan
interiation no  1 

is nan
interiation no  2 

is nan
interiation no  3 

is nan
interiation no  4 

is nan
interiation no  5 

is nan
interiation no  6 

is nan
interiation no  7 

is nan
interiation no  8 

is nan
interiation no  9 

is nan
interiation no  10 

is nan
[49, 52, 41, 0, 55, 22, 32, 96, 16, 21, 56]


In [19]:
scipy.io.savemat('/home/hades/codes/leiden/temp/best_frame_sunny.mat', mdict={'central': best_frame})

In [20]:
final_Clusters[i][np.argmax(np.array([1,2,3]))]

429

In [21]:

descriptor=0
total_score=np.empty((0,0),int)
mat=np.empty((0,0),int)

### **Saving characteristic Frames from each dataset**###

In [22]:
np.save('/home/hades/codes/leiden/cold/cendroids_dataset_%s_bestFrame' %dataset ,best_frame)
np.save('/home/hades/codes/leiden/cold/cendroids_dataset_%s_bestFrame_descriptor.npy' %dataset ,best_frame_descriptor)
np.save('/home/hades/codes/leiden/cold/cendroids_dataset_%s_bestFrame_local' %dataset ,best_frame_local)

In [23]:
np.save('/home/hades/codes/leiden/cold/%s_clusters.npy'%dataset ,final_Clusters, allow_pickle=True)

In [24]:

if dataset !='cloudy':
    ytf

mat=0
descriptor=0
sys.getsizeof(final_Clusters)

192

In [25]:
os.chdir("/home/hades/codes/leiden/cold")
sunny_best_frame=np.load('cendroids_dataset_sunny_bestFrame.npy',allow_pickle=True)
sunny_best_descriptor=np.load('/home/hades/codes/leiden/cold/cendroids_dataset_sunny_bestFrame_descriptor.npy',allow_pickle=True)
cloudy_best_frame=np.load('cendroids_dataset_cloudy_bestFrame.npy',allow_pickle=True)
cloudy_best_descriptor=np.load('/home/hades/codes/leiden/cold/cendroids_dataset_cloudy_bestFrame_descriptor.npy',allow_pickle=True)
night_best_frame=np.load('cendroids_dataset_night_bestFrame.npy',allow_pickle=True)
night_best_descriptor=np.load('/home/hades/codes/leiden/cold/cendroids_dataset_night_bestFrame_descriptor.npy',allow_pickle=True)

### **Matching clusters to cloudy (using cendroids)**###

**characteristic frames of cloudy dataset:**

In [26]:
print(cloudy_best_frame)
print(sunny_best_frame)
from scipy.spatial import distance
distance.euclidean(sunny_best_descriptor[1],cloudy_best_descriptor[0])

[ 218 1425  365  919  764  708  946   53  561]
[ 438   52 1325 1584  545  155 1223  869 1345  309 1463]


1.094031826842451

### Splitting cloudy images to new folders

In [27]:
import shutil, os
try:
    os.chdir("/home/hades/codes/leiden/cold/dataset_cloudy/")
    files = []
    name='img00000000'
    Im=230
    #os.mkdir('/home/hades/codes/leiden/cold/class_test1/')
    for i in range (0,len(final_Clusters)):
        #os.rmdir('/home/hades/codes/leiden/cold/class/%d'%i)
        os.mkdir('/home/hades/codes/leiden/cold/class_test1/%d'%i)
        files=[]
        for  Im in final_Clusters[i]:
            #name='img00000000'
            #name=name[0:-len(str(Im+1))]+str(Im+1)
            name=str(Im)
            files.append(name)
            print(name)
    
            
           
            for f in files:
             
                shutil.copy("%s.jpeg"%f, '/home/hades/codes/leiden/cold/class_test1/%d'%i)
except FileExistsError:
    print("already exist, skipping..")

already exist, skipping..


### Extracting descriptors from cloudy images

In [28]:
#strt!@

import cv2   
import glob
desc=[]
for i in range(0,len(final_Clusters)):
    os.chdir("/home/hades/codes/leiden/cold/class_test1/%d" %i)
    desc.append([])
    for filename in glob.glob('*.jpeg'):
        img = cv2.imread(filename,0)
        #cv2.imshow("wer",img)
        #cv2.waitKey(30)
        
        cv2.destroyAllWindows()
        sift=cv2.xfeatures2d.SURF_create(hessianThreshold = 200)#6000
# Find keypoints and descriptors directly
        kp, des = sift.detectAndCompute(img,None)
        desc[i].append(des)
        #print("\n")
#print(desc)
#print("fvdfvd")
np.save('/home/hades/codes/leiden/cold/desc_small_temp2.npy',desc)

In [29]:
desc=np.load('/home/hades/codes/leiden/cold/desc_small_temp2.npy',allow_pickle=True)

### Creating descriptor pool for cloudy 

In [30]:
len(d_pool[0])

NameError: name 'd_pool' is not defined

In [34]:
d_pool=[]
cl_idx=[]
counter=0
for i in range(0,len(final_Clusters)):
    #d_pool.append([])
    for j in range(0,len(desc[i])):
        #d_pool[i].append([])
        if desc[i][j] is None:
            continue
        for k in range(0,len(desc[i][j])):
            d_pool.append(desc[i][j][k])
            counter+=1
    cl_idx.append(counter)
        
#d_pool   

In [0]:
np.save('/home/hades/codes/leiden/cold/case_study/d_idx.y'np,cl_idx)

In [35]:
np.save('/home/hades/codes/leiden/temp/d_pool_cold.npy',d_pool)
np.save('/home/hades/codes/leiden/temp/cl_idx_cold.npy',cl_idx)

In [0]:
gt_night=[2,1,0,4,6,3,5,5]
gt_sunny=[3,4,0,2,6,5,0]

### Night Re-loc

In [0]:
print(filename)
img = cv2.imread(filename,0)
print(len(img))
sift=cv2.xfeatures2d.SURF_create(2000)
kp, des = sift.detectAndCompute(img,None)
print(sift)

In [0]:
####test with knn
d_night=[]
counter=0

os.chdir("/home/hades/codes/leiden/cold/dataset_night/")

for i in night_best_frame:

    for filename in glob.glob('*.jpeg'):
        if int(filename[:-5])==i:
            d_night.append([])
            img = cv2.imread(filename,0)
            sift=cv2.xfeatures2d.SURF_create(200)
# Find keypoints and descriptors directly
            kp, des = sift.detectAndCompute(img,None)
            des.any()==[None]
            d_night[counter].append(des)
            counter+=1
            print("\n")
#print(d_sunny)



d_pool_night=[]
for i in range(0,len(night_best_frame)):
    d_pool_night.append([])
    for j in range(0,len(d_night[i])):
        #d_pool_sunny[i].append([])
      
        for k in range(0,len(d_night[i][j])):
            d_pool_night[i].append(d_night[i][j][k])
        
        
len(d_pool_night[1])

local_frame=np.load('/home/hades/codes/leiden/cold/cendroids_dataset_night_bestFrame_local.npy')
neigh = NearestNeighbors(n_neighbors=2)
neigh.fit(d_pool) 
vote=[]
for i in range(0,len(night_best_frame)):
    match=neigh.kneighbors(np.asarray(d_pool_night[i]))#.reshape(1, -1))
    #print(match)
    vote.append([])
    for x in range(0,len(cl_idx)):
        if match[1][0][0]<cl_idx[x]:
            vote[i].append(x)
            break
#print(vote)
            
#np.save('/home/hades/codes/leiden/cold/vote_night.npy', vote)
vote2=[]
for i in range(0,len(vote)):
    m=0
    vote2.append([])
    for j in range(0,len(vote[i])):
       # print('night[%d] --> cloudy [%d]'%(i,(len(vote[i][j]))))
        vote2[i].append(len(vote[i]))
success=0
night_match=[]
for i in range(0,len(vote)):    
    print('cloudy[%d] --> night[%d]'%((vote[i][0],i)))


    
#closest=neigh.kneighbors(descriptor_test.reshape(1, -1))

In [0]:
del vote
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(np.array(d_pool)) 
print('i am here!')
match=[]
#for i in range(0,len(best_frame)):
#    print('loop',i)
#    match.append(neigh.kneighbors(np.asarray(features[i])))#.reshape(1, -1))
#    print(match[i])
vote=[]
idx2=[]
print(cl_idx)
#print(idx2)
for i in range(0,len(night_best_frame)):
    match=neigh.kneighbors(np.asarray(d_pool_night[i]))#.reshape(1, -1))
    #print(match)
    vote.append([])
    for j in range(0,len(match[1])):
        for x in range(0,len(cl_idx)):
            if match[1][j]<cl_idx[x]:
                vote[i].append(x)
                break

In [0]:
def most_frequent(List): 
    dict = {} 
    count, itm = 0, '' 
    for item in reversed(List): 
        dict[item] = dict.get(item, 0) + 1
        if dict[item] >= count : 
            count, itm = dict[item], item 
    return(itm) 
  

for i in range(0,len(vote)):
    print(most_frequent(vote[i])) 
os.chdir("/home/hades/codes/leiden/cold/case_study")    
np.save('night_match.npy',vote)      



In [0]:
vote

### Sunny Re-loc

In [0]:
####test with knn
d_sunny=[]
counter=0

os.chdir("/home/hades/codes/leiden/cold/dataset_sunny/")

for i in sunny_best_frame:

    for filename in glob.glob('*.jpeg'):
        if int(filename[:-5])==i:
            d_sunny.append([])
            img = cv2.imread(filename,0)
            sift=cv2.xfeatures2d.SURF_create(hessianThreshold = 200)
# Find keypoints and descriptors directly
            kp, des = sift.detectAndCompute(img,None)
            d_sunny[counter].append(des)
            counter+=1
            print("\n")
#print(d_sunny)


d_pool_sunny=[]
for i in range(0,len(sunny_best_frame)):
    d_pool_sunny.append([])
    for j in range(0,len(d_sunny[i])):
        #d_pool_sunny[i].append([])
        for k in range(0,len(d_sunny[i][j])):
            d_pool_sunny[i].append(d_sunny[i][j][k])
        
        
len(d_pool_sunny[1])

local_frame=np.load('/home/hades/codes/leiden/cold/cendroids_dataset_sunny_bestFrame_local.npy')
neigh = NearestNeighbors(n_neighbors=2)
neigh.fit(d_pool) 
vote=[]
for i in range(0,len(sunny_best_frame)):
    match=neigh.kneighbors(np.asarray(d_pool_sunny[i]))#.reshape(1, -1))
    #print(match)
    vote.append([])
    for x in range(0,len(cl_idx)):
        if match[1][0][0]<cl_idx[x]:
            vote[i].append(x)
            break
print(vote)
            
#np.save('/home/hades/codes/leiden/cold/vote_night.npy', vote)
vote2=[]
for i in range(0,len(vote)):
    m=0
    vote2.append([])
    for j in range(0,len(vote[i])):
       # print('night[%d] --> cloudy [%d]'%(i,(len(vote[i][j]))))
        vote2[i].append(len(vote[i]))
success=0

for i in range(0,len(vote)):    
    print('cloudy[%d] --> sunny[%d]'%((vote[i][0],i)))

#closest=neigh.kneighbors(descriptor_test.reshape(1, -1))

In [0]:
del vote
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(np.array(d_pool)) 
print('i am here!')
match=[]
#for i in range(0,len(best_frame)):
#    print('loop',i)
#    match.append(neigh.kneighbors(np.asarray(features[i])))#.reshape(1, -1))
#    print(match[i])
vote=[]
idx2=[]
print(cl_idx)
#print(idx2)
for i in range(0,len(sunny_best_frame)):
    match=neigh.kneighbors(np.asarray(d_pool_sunny[i]))#.reshape(1, -1))
    #print(match)
    vote.append([])
    for j in range(0,len(match[1])):
        for x in range(0,len(cl_idx)):
            if match[1][j]<cl_idx[x]:
                vote[i].append(x)
                break

In [0]:
def most_frequent(List): 
    dict = {} 
    count, itm = 0, '' 
    for item in reversed(List): 
        dict[item] = dict.get(item, 0) + 1
        if dict[item] >= count : 
            count, itm = dict[item], item 
    return(itm) 
  

for i in range(0,len(vote)):
    print(most_frequent(vote[i])) 
os.chdir("/home/hades/codes/leiden/cold/case_study")    
np.save('sunny_match.npy',vote)     



In [0]:
import glob
#feature vote onlyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
d_pool=np.load('/home/hades/codes/leiden/temp/d_pool_cold.npy')
cl_idx=np.load('/home/hades/codes/leiden/temp/cl_idx_cold.npy')

d_sunny=[]
counter=0

os.chdir("/home/hades/codes/leiden/cold/dataset_sunny/")
for j in range(0,len(final_Clusters)):
    d_sunny.append([])
    for i in final_Clusters[j]:

        for filename in glob.glob('*.jpeg'):
            if int(filename[:-5])==i:
                img = cv2.imread(filename,0)
                sift=cv2.xfeatures2d.SURF_create(200)
    # Find keypoints and descriptors directly
                kp, des = sift.detectAndCompute(img,None)
                des.any()==[None]
                d_sunny[j].append(des)
                counter+=1
                #print("\n")
    #print(d_sunny)






    #print(d_sunny)

d_pool_sunny=[]
for i in range(0,len(final_Clusters)):
    d_pool_sunny.append([])
    for j in range(0,len(d_sunny[i])):
        #d_pool_sunny[i].append([])
        for k in range(0,len(d_sunny[i][j])):
            d_pool_sunny[i].append(d_sunny[i][j][k])
        
        
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(np.array(d_pool)) 
print('i am here!')
match=[]
#for i in range(0,len(best_frame)):
#    print('loop',i)
#    match.append(neigh.kneighbors(np.asarray(features[i])))#.reshape(1, -1))
#    print(match[i])
vote=[]
idx2=[]
print(cl_idx)
#print(idx2)
for i in range(0,len(final_Clusters)):
    match=neigh.kneighbors(np.asarray(d_pool_sunny[i]))#.reshape(1, -1))
    #print(match)
    vote.append([])
    for j in range(0,len(match[1])):
        for x in range(0,len(cl_idx)):
            if match[1][j]<cl_idx[x]:
                vote[i].append(x)
                break
def most_frequent(List): 
    dict = {} 
    count, itm = 0, '' 
    for item in reversed(List): 
        dict[item] = dict.get(item, 0) + 1
        if dict[item] >= count : 
            count, itm = dict[item], item 
    return(itm) 
  

for i in range(0,len(vote)):
    print(most_frequent(vote[i])) 

#print(vote[0])


In [15]:
import cv2
import glob
#feature vote onlyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy
d_pool=np.load('/home/hades/codes/leiden/temp/d_pool_cold.npy')
cl_idx=np.load('/home/hades/codes/leiden/temp/cl_idx_cold.npy')

d_night=[]
counter=0

os.chdir("/home/hades/codes/leiden/cold/dataset_night/")
for j in range(0,len(final_Clusters)):
    d_night.append([])
    for i in final_Clusters[j]:

        for filename in glob.glob('*.jpeg'):
            if int(filename[:-5])==i:
                img = cv2.imread(filename,0)
                sift=cv2.xfeatures2d.SURF_create(200)
    # Find keypoints and descriptors directly
                kp, des = sift.detectAndCompute(img,None)
                des.any()==[None]
                d_night[j].append(des)
                counter+=1
                #print("\n")
    #print(d_sunny)






    #print(d_sunny)

d_pool_night=[]
for i in range(0,len(final_Clusters)):
    d_pool_night.append([])
    for j in range(0,len(d_night[i])):
        #d_pool_sunny[i].append([])
        for k in range(0,len(d_night[i][j])):
            d_pool_night[i].append(d_night[i][j][k])
        
        
neigh = NearestNeighbors(n_neighbors=1)
neigh.fit(np.array(d_pool)) 
print('i am here!')
match=[]
#for i in range(0,len(best_frame)):
#    print('loop',i)
#    match.append(neigh.kneighbors(np.asarray(features[i])))#.reshape(1, -1))
#    print(match[i])
vote=[]
idx2=[]
print(cl_idx)
#print(idx2)
for i in range(0,len(final_Clusters)):
    match=neigh.kneighbors(np.asarray(d_pool_night[i]))#.reshape(1, -1))
    #print(match)
    vote.append([])
    for j in range(0,len(match[1])):
        for x in range(0,len(cl_idx)):
            if match[1][j]<cl_idx[x]:
                vote[i].append(x)
                break
def most_frequent(List): 
    dict = {} 
    count, itm = 0, '' 
    for item in reversed(List): 
        dict[item] = dict.get(item, 0) + 1
        if dict[item] >= count : 
            count, itm = dict[item], item 
    return(itm) 
  

for i in range(0,len(vote)):
    print(most_frequent(vote[i])) 

#print(vote[0])


AttributeError: 'NoneType' object has no attribute 'any'

### ---TOOLS---

In [0]:
import shutil, os
dataset='night'
import glob
files0=glob.glob("/home/hades/Downloads/Datasets/seq1_%s1/std_cam/*.jpeg"%dataset)
files0.sort(reverse=False);
print(len(files0))
counter5=0
os.mkdir('/home/hades/codes/leiden/cold/dataset_%s/'%dataset)
for i in files0:
    shutil.copy("%s"%i, '/home/hades/codes/leiden/cold/dataset_%s/%d.jpeg'%(dataset,counter5))
    counter5+=1

In [0]:
print(d_pool_night[294])

In [0]:
np.argwhere(np.asarray(final_Clusters[0])==1225)

In [0]:
counter=0
print(night_best_frame)
print(sunny_best_frame)
#print(cloudy_best_frame)
files0=glob.glob("/home/hades/Downloads/Datasets/seq1_night1/std_cam/*.jpeg")
files0.sort(reverse=False);
for i in night_best_frame:
    counter+=1
    img=cv2.imread(files0[i])
    cv2.imshow("cluster %s frame %s"%(counter-1,str(i)),img)
    
cv2.waitKey(0) 
cv2.destroyAllWindows() 
files0=glob.glob("/home/hades/Downloads/Datasets/seq1_sunny1/std_cam/*.jpeg")
files0.sort(reverse=False);
counter=0
for i in sunny_best_frame:
    counter+=1
    img=cv2.imread(files0[i])
    cv2.imshow("cluster %s frame %s"%(counter-1,str(i)),img)
    
cv2.waitKey(0) 
cv2.destroyAllWindows() 

In [0]:
np.save('/home/hades/codes/leiden/cold/vote_night.npy' ,final_Clusters, allow_pickle=True)